# Save, Load and Export Models in Keras

## Task 1: Import Libraries

In [31]:
import tensorflow as tf
import numpy as np
import os

print('This notebook works with TensorFlow version:', tf.__version__)

folders = ['tmp', 'models', 'model_name', 'weights']
for folder in folders:
    if not os.path.isdir(folder):
        os.mkdir(folder)

print(os.listdir('.'))

This notebook works with TensorFlow version: 2.3.0
['.config', 'weights', 'model_name', 'tmp', 'models', 'sample_data']


## Task 2: Create Model

In [32]:
def create_model():
    model = tf.keras.models.Sequential([
                                        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,)),
                                        tf.keras.layers.Dense(128, activation='relu'),
                                        tf.keras.layers.Dense(10, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

    return model

model = create_model()
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_25 (Dense)             (None, 128)               16512     
_________________________________________________________________
dense_26 (Dense)             (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


## Task 3: Data Preprocessing

In [33]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
X_train = np.reshape(X_train, (X_train.shape[0], 784)) / 255.
X_test = np.reshape(X_test, (X_test.shape[0], 784)) / 255.

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

## Task 4: Model Checkpoint During Training

In [34]:
checkpoint_dir = '/content/weights/'
_ = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=2, batch_size=512,
    callbacks=[
               tf.keras.callbacks.ModelCheckpoint(
                   os.path.join(checkpoint_dir, 'epoch_{epoch:02d}__acc__{val_acc:.4f}'),
                   monitor = 'val_acc',
                   save_weights_only=True,
                   save_best_only=True
               )
    ]
)

Epoch 1/2
118/118 [==============================] - 1s 11ms/step - loss: 0.6722 - acc: 0.7729 - val_loss: 0.4956 - val_acc: 0.8240
Epoch 2/2
118/118 [==============================] - 1s 9ms/step - loss: 0.4384 - acc: 0.8462 - val_loss: 0.4386 - val_acc: 0.8422


In [35]:
os.listdir(checkpoint_dir)

['epoch_02__acc__0.8422.data-00000-of-00001',
 'epoch_01__acc__0.8240.index',
 'epoch_02__acc__0.8422.index',
 'checkpoint',
 'epoch_01__acc__0.8240.data-00000-of-00001']

## Task 5: Load Weights

In [36]:
model_new = create_model()
print(model_new.evaluate(X_test, y_test, verbose=False))

[2.3637712001800537, 0.1005999967455864]


In [37]:
model_new.load_weights('/weights/epoch_02__acc__0.8403')
print(model_new.evaluate(X_test, y_test, verbose=False))

[0.44820472598075867, 0.8403000235557556]


## Task 6: Saving Complete Model During Training

In [38]:
models_dir = '/content/models'

_ = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=2, batch_size=512,
    callbacks=[
               tf.keras.callbacks.ModelCheckpoint(
                   os.path.join(models_dir, 'epoch_{epoch:02d}__acc__{val_acc:.4f}.h5'),
                   monitor = 'val_acc',
                   save_weights_only=False,
                   save_best_only=False
               )
    ]
)

Epoch 1/2
118/118 [==============================] - 1s 10ms/step - loss: 0.3910 - acc: 0.8608 - val_loss: 0.4324 - val_acc: 0.8492
Epoch 2/2
118/118 [==============================] - 1s 10ms/step - loss: 0.3641 - acc: 0.8709 - val_loss: 0.4087 - val_acc: 0.8554


In [39]:
os.listdir(models_dir)

['epoch_01__acc__0.8492.h5',
 'epoch_02__acc__0.8732.h5',
 'epoch_02__acc__0.8554.h5',
 'epoch_01__acc__0.8707.h5']

## Task 7: Load Models

In [41]:
model = create_model()
print(model.evaluate(X_test, y_test, verbose=False))

[2.460688591003418, 0.09229999780654907]


In [44]:
model = tf.keras.models.load_model('/content/models/epoch_01__acc__0.8707.h5')
print(model.evaluate(X_test, y_test, verbose=False))
model.summary()

[0.3637693226337433, 0.8707000017166138]
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 128)               100480    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1290      
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


## Task 8: Manually Saving Weights and Models

In [45]:
model.save_weights('/content/tmp/manually_saved.w')
os.listdir('/content/tmp')

['manually_saved.w.data-00000-of-00001',
 'checkpoint',
 'manually_saved.w.index',
 'manually_saved.h5']

In [46]:
model.save('tmp/manually_saved.h5')
os.listdir('/content/tmp')

['manually_saved.w.data-00000-of-00001',
 'checkpoint',
 'manually_saved.w.index',
 'manually_saved.h5']

## Task 9: Exporting and Restoring SavedModel Format

In [47]:
model.save('/content/model_name')
os.listdir('/content/model_name')

INFO:tensorflow:Assets written to: /content/model_name/assets


['assets', 'variables', 'saved_model.pb']

In [48]:
model = tf.keras.models.load_model('/content/model_name')
print(model.evaluate(X_test, y_test, verbose=False))


[0.3637693226337433, 0.8707000017166138]
